In [ ]:
import numpy as np
import cv2
import json
from google.colab.patches import cv2_imshow
import pprint
import os
import random
from PIL import Image
import copy
pp = pprint.PrettyPrinter(indent=1)
extra=0

In [ ]:
ppath=os.listdir('drive/MyDrive/TRTPose/Extra/dataset')
print(len(ppath))
npath=os.listdir('drive/MyDrive/TRTPose/Extra/neg_dataset')
print(len(npath))

323
487


In [ ]:
f_json={'images':[],'annotations':[],'categories': [{'color': '#34d34d',
                 'id': 1,
                 'keypoint_colors': ['#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000'],
                 'keypoints': ['top_front_left',
                               'top_front_right',
                               'top_back_left',
                               'top_back_right',
                               'bot_front_left',
                               'bot_front_right',
                               'bot_back_left',
                               'bot_back_right'],
                 'metadata': {},
                 'name': 'forklift',
                 'skeleton': [[1, 2],
                              [1, 3],
                              [1, 5],
                              [2, 4],
                              [2, 6],
                              [3, 7],
                              [3, 4],
                              [4, 8],
                              [5, 6],
                              [5, 7],
                              [6, 8],
                              [7, 8]],
                 'supercategory': ''},
                {'color': '#c28e00',
                 'id': 3,
                 'keypoint_colors': ['#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000'],
                 'keypoints': ['nose',
                               'left_eye',
                               'right_eye',
                               'left_ear',
                               'right_ear',
                               'left_shoulder',
                               'right_shoulder',
                               'left_elbow',
                               'right_elbow',
                               'left_wrist',
                               'right_wrist',
                               'left_hip',
                               'right_hip',
                               'left_knee',
                               'right_knee',
                               'left_ankle',
                               'right_ankle'],
                 'metadata': {},
                 'name': 'person',
                 'skeleton': [[1, 2],
                              [1, 3],
                              [2, 3],
                              [2, 4],
                              [3, 5],
                              [4, 6],
                              [5, 7],
                              [6, 12],
                              [6, 7],
                              [6, 8],
                              [7, 13],
                              [7, 9],
                              [8, 10],
                              [9, 11],
                              [12, 14],
                              [12, 13],
                              [13, 15],
                              [14, 16],
                              [15, 17]],
                 'supercategory': ''}]}

In [ ]:
f = open('drive/MyDrive/TRTPose/Extra/new_Data.json')
data = json.load(f)

In [ ]:
def get_concat_h_blank(im1, im2, color=(128, 128, 128)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v_blank(im1, im2, color=(128, 128, 128)):
    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

In [ ]:
def update_json(nam,filename,height,width,idd,imdata):
  tick1=False
  for na in range(0,len(nam)):
    if(not('neg' in nam[na])):
      tick1=True
      for img in data['images']:
        if(img['file_name']==nam[na]):
          x,y=0,0
          if(na==0):
            x,y=0,0
          if(na==1):
            x=int(imdata[0][0])
            y=0
          if(na==2):
            x=0
            y=int(max(imdata[0:2,1]))
          if(na==3):
            x=int(imdata[2][0])
            y=int(max(imdata[0:2,1]))
          tick2=False
          for ann in data['annotations']:
            if(ann['image_id']==img['id'] and ann['category_id']==1):
              ann_copy = copy.deepcopy(ann)
              seg=ann_copy['segmentation']
              
              for i in range(0,len(seg)):
                se=np.array(seg[i],dtype=np.int32)
                se=np.resize(se, (len(se)//2,2))
                se=se+(x,y)
                se=np.resize(se, len(se)*2)
                ann_copy['segmentation'][i]=[float(i) for i in list(se)]
              kp=ann_copy['keypoints']
              kp=np.array(kp,dtype=np.int32)
              kp=np.resize(kp, (len(kp)//3,3))
              kp=kp+(x,y,0)
              kp=np.resize(kp, len(kp)*3)
              ann_copy['keypoints']=[float(i) for i in list(kp)]
              ann_copy['bbox'][0]=ann_copy['bbox'][0]+x
              ann_copy['bbox'][1]=ann_copy['bbox'][1]+y
              ann_copy['image_id']=int(idd)
              ann_copy['id']=idd*1000+na
              tick2=True
              f_json['annotations'].append(ann_copy)
            if(ann['image_id']==img['id'] and ann['category_id']==3):
              ann_copy = copy.deepcopy(ann)
              seg=ann_copy['segmentation']
              
              for i in range(0,len(seg)):
                se=np.array(seg[i],dtype=np.int32)
                se=np.resize(se, (len(se)//2,2))
                se=se+(x,y)
                se=np.resize(se, len(se)*2)
                ann_copy['segmentation'][i]=[float(i) for i in list(se)]
              kp=ann_copy['keypoints']
              kp=np.array(kp,dtype=np.int32)
              kp=np.resize(kp, (len(kp)//3,3))
              kp=kp+(x,y,0)
              kp=np.resize(kp, len(kp)*3)
              ann_copy['keypoints']=[float(i) for i in list(kp)]
              ann_copy['bbox'][0]=ann_copy['bbox'][0]+x
              ann_copy['bbox'][1]=ann_copy['bbox'][1]+y
              ann_copy['image_id']=int(idd)
              ann_copy['id']=idd*1001+na
              tick2=True
              f_json['annotations'].append(ann_copy)

          if(tick2==False):
            print('ann is missing',img['id'])


  img_d={'id'	:	idd,
  'dataset_id'	:	10,
  'path'	:	'/datasets/merged_data/'+filename,
  'width'	:	width,
  'height'	:	height,
  'file_name'	:	filename}
  f_json['images'].append(img_d) 
  if(tick1==False):
    print('img missing',na)

In [ ]:
elist=[]
num=0
while(num<800):
  tick=True
  while(tick==True):
    iarea=[]
    sem=random.randint(2, 4)
    nam=[]
    nam1=[]
    imdata=[]
    for i in random.sample(ppath, sem):
      nam.append(i)
      nam1.append(i)
      im=Image.open('drive/MyDrive/TRTPose/Extra/combined_data/'+i)
      width, height = im.size
      area=width*height
      iarea.append(area)
    for i in random.sample(npath, 4-sem):
      nam.append(i)
    std=np.std(iarea)
    if((not(set(nam1) in elist)) and std<2000):
      tick=False
  num=num+1
  random.shuffle(nam)
  elist.append(set(nam1))
  imlist=[]
  for i in nam:
    im=Image.open('drive/MyDrive/TRTPose/Extra/combined_data/'+i)
    imlist.append(im)
    imdata.append([im.width, im.height])
  img1=get_concat_h_blank(imlist[0],imlist[1])
  img2=get_concat_h_blank(imlist[2],imlist[3])
  img=get_concat_v_blank(img1,img2)
  img.save('drive/MyDrive/TRTPose/Extra/merged_images/'+str(num)+'_merged.jpg')
  update_json(nam,filename=str(num)+'_merged.jpg',height=img.height, width=img.width, idd=num+0,imdata=np.array(imdata))

KeyboardInterrupt: ignored

In [ ]:
print(len(f_json['annotations']))

1658


In [ ]:
pp.pprint(f_json)

In [ ]:
with open("drive/MyDrive/TRTPose/Extra/merge_data.json", "w") as outfile:
    json.dump(f_json, outfile)

In [ ]:
path=os.listdir('drive/MyDrive/TRTPose/Extra/merged_images/')
print(len(path))

719


In [ ]:
for i in path:
  im1=cv2.imread('drive/MyDrive/TRTPose/Extra/merged_images/'+i)
  for j in path:
    im2=cv2.imread('drive/MyDrive/TRTPose/Extra/merged_images/'+j)
    if im1.shape == im2.shape:
      if(i==j):
        continue
      print("The images have same size and channels")
      difference = cv2.subtract(im1, im2)
      b, g, r = cv2.split(difference)
      if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        print("The images are completely Equal")

The images have same size and channels
The images have same size and channels
The images have same size and channels
The images have same size and channels
The images have same size and channels
The images have same size and channels
